# Speech Emotion Recognization

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Importing necessary libaries

In [1]:
import librosa
import soundfile
import os, pickle
import glob as gb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

## DataFlare Extract Features

In [2]:
def extract_features(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X=sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        result=np.array([])
        if chroma:
          chroma=np.mean(librosa.feature.chroma_stft(y=X, sr=sample_rate).T,axis=0)
          result=np.hstack((result,chroma))
        if mfcc:
          mfcc=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
          result=np.hstack((result,mfcc))
        if mel:
          mel=np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
          result=np.hstack((result,mel))
    return result


## Defining a dictionary. DataFlare - Emotions in the RAVDESS dataset

In [3]:
emotions = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}

### Emotions to Observe

In [4]:
observed_emotions=['calm','happy','fearful','disgust']

## Loading a dataset

In [5]:
def load_data(test_size=0.2):
  x,y=[],[]
  for file in gb.glob("/content/drive/MyDrive/Data for ML(rough)/Dataset/Actor_*/*.wav"):
    file_name=os.path.basename(file)
    emotion=emotions[file_name.split("-")[2]]
    if emotion not in observed_emotions:
      continue
    feature=extract_features(file,mfcc=True, chroma=True, mel=True)
    x.append(feature)
    y.append(emotion)
  return train_test_split(np.array(x),y,test_size=test_size,random_state=9)

## Splitting the dataset into Train and Test set

In [6]:
x_train,x_test,y_train,y_test=load_data(test_size=0.20)
x_train

array([[4.65418577e-01, 4.91756201e-01, 5.50340891e-01, ...,
        3.38972472e-06, 3.47467108e-06, 3.59502678e-06],
       [4.77691472e-01, 5.40696502e-01, 5.55202603e-01, ...,
        4.30930669e-07, 4.01358051e-07, 4.06331111e-07],
       [5.66166580e-01, 4.81962085e-01, 3.89366806e-01, ...,
        4.05141982e-05, 4.78639449e-05, 2.69214725e-05],
       ...,
       [3.80818903e-01, 4.23536003e-01, 4.61970836e-01, ...,
        6.24168606e-05, 5.40789042e-05, 3.72418690e-05],
       [3.77233565e-01, 3.22094858e-01, 3.15086007e-01, ...,
        2.97792023e-04, 1.04964238e-04, 9.26514404e-05],
       [2.65766531e-01, 2.93289483e-01, 3.57265115e-01, ...,
        5.19565656e-04, 6.20016071e-04, 4.71762236e-04]])

In [7]:
print((x_train.shape[0], x_test.shape[0]))

(614, 154)


## Features extracted

In [8]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


## Initialize the Multi Layer Perceptron Classifier

In [9]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(400,), learning_rate='adaptive', max_iter=500)

## Train the model

In [10]:
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(400,),
              learning_rate='adaptive', max_iter=500)

## Prediction for the test set

In [11]:
y_pred=model.predict(x_test)

## Calculating the accuracy of our model

In [12]:
accuracy=accuracy_score(y_true=y_test,y_pred=y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 77.27%
